In [3]:
%load_ext autoreload
%autoreload 2
%load_ext jupyternotify
import numpy as np 
from sets import Set
from conf_ope_rl import * 
from __future__ import division 

pu1 = 1/6
pu2 = 1/2
pis1u1 = 1.0/4
pis2u1 = 1.0/6

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


## Stationary distribution 
$p^{\infty}_b(s,u)  $ 

What is $p^{\infty}_b(s,a,s')$? 
Note $\mathbb{E}[\pi_b(a \mid s,u)] = \frac{1}{2} \pi_b(a=1\mid s,u=1) +  \frac{1}{2} \pi_b(a=1\mid s,u=2) $

The true marginalized transition matrix is $p_b^\infty(s,a,s') = \mathbb{E}_{u \sim p_b}[ P(s' \mid a,s,u) \pi_b(a \mid s,u) p_b^\infty(s,u)] $

and 
$p_b^\infty(s',a\mid s) = \mathbb{E}_{u \sim p_b}[ P(s' \mid a,s,u) \pi_b(a \mid s,u) p_b^\infty(u \mid s)] $

The biased transitions are $\tilde p(s'\mid s,a) = p_b^\infty(s',a\mid s) /  \mathbb{E}_{u\sim p_b^\infty} [\pi_b(a \mid s,u)\mid s] $


In [4]:
from __future__ import division
nS = 4
nA = 2
P = np.zeros([nS,nA, nS])
state_dist = np.ones(nS) * 1.0/nS
n = 20000
nU = 2
nSmarg = nS / nU
gamma = 0.99
p0 = np.asarray([0.25, 0.25,0.25,0.25])
p_e_su = np.asarray([ p0 ,1-p0 ])
print p_e_su

# very extreme


def get_P(pu1, pu2, pis1u1, pis2u1): 
    Pi = np.array(# s=a,u1; s=a,u2; s=b,u1; s=b,u2
        [[pis1u1, 1-pis1u1, pis2u1, 1-pis2u1],
            [1-pis1u1, pis1u1, 1-pis2u1, pis2u1]]) 
    P = np.zeros([nS,nA, nS])
    P[:,0,:] = np.asarray([np.asarray([pu1, 
         pu1, (1/2 - pu1), (1/2 - pu1)]), np.asarray([(1/2 - pu2), (1/2 - pu2), pu2, pu2]), 
    np.asarray([pu1, 
         pu1, (1/2 - pu1), (1/2 - pu1)]), np.asarray([(1/2 - pu2), (1/2 - pu2), pu2, pu2])]);
    P[:,1,:] = np.asarray([np.asarray([(1/2 - pu1), (1/2 - pu1), pu1, pu1]), 
    np.asarray([pu2, 
         pu2, (1/2 - pu2), (1/2 - pu2)]), np.asarray([(1/2 - pu1), (1/2 - pu1), pu1, pu1]), 
    np.asarray([pu2, pu2, (1/2 - pu2), (1/2 - pu2)])]);
    return [P,Pi]

[P,Pi] = get_P(pu1, pu2, pis1u1, pis2u1)

[[0.25 0.25 0.25 0.25]
 [0.75 0.75 0.75 0.75]]


In [5]:
tight = False
# def get_full_agg_w_densratio( gamma, nS, nA, nU, P, Pi, p_e_su, state, n ): 
nSmarg = int(nS / nU)
[ stateChangeHist, stateHist, a_s, s_a_sprime, p_infty_b_su, distr_hist ] =  simulate_rollouts( 
    nS, nA, P, Pi, state_dist, n )
p_infty_b_s = (reshape_byxrow(p_infty_b_su.T, nU).T).flatten()
p_infty_b_su = p_infty_b_su.flatten()
print p_infty_b_su, p_infty_b_s
print('stationary distn on s,u: ', p_infty_b_su)

# process history 
[ p_a1_su, joint_s_a_sprime, s_a_giv_sprime ] = get_auxiliary_info_from_traj(stateChangeHist, 
                                        stateHist, a_s, s_a_sprime, p_infty_b_su, distr_hist, nA,nS)
# agg history and process
[aggStateHist, p_a1_s, p_e_s, agg_s_a_sprime, joint_s_a_sprime_agg, s_a_giv_sprime_agg] = agg_history(
            stateHist, s_a_sprime, p_infty_b_s, a_s, p_e_su, nA, nS, nSmarg, nU)
# Simulate from evaluation policy: 
[ stateChangeHist_e, stateHist_e, a_s_e, s_a_sprime_e, p_infty_e, distr_hist_e ] =  simulate_rollouts( 
    nS, nA, P, p_e_su, state_dist, n )


# Solve on the full information space 
w_su = get_w_lp(gamma, s_a_giv_sprime, p_infty_b_su, p_e_su, p_a1_su, nA, nS, tight = tight)
print 'LP solution norm', w_su / sum(w_su)
print w_su
print 'oracle solution norm', (p_infty_e / p_infty_b_su) / np.sum((p_infty_e / p_infty_b_su))
print (p_infty_e / p_infty_b_su)

w_su_oracle_norm = (p_infty_e / p_infty_b_su)/ np.sum((p_infty_e / p_infty_b_su))
w_s_oracle_norm = reshape_byxrow(w_su_oracle_norm.T,nU).T.flatten()
print 'marginalized stationary distribution', w_s_oracle_norm





[0.25 0.25 0.25 0.25]
[0.20318984 0.20183991 0.29733513 0.29763512] [0.40502975 0.59497025]
('stationary distn on s,u: ', array([0.20318984, 0.20183991, 0.29733513, 0.29763512]))
[0.25 0.25 0.25 0.25]

--------------------------------------------
--------------------------------------------

Using license file /Users/az/gurobi.lic
Academic license - for non-commercial use only
LP solution norm [0.37013753 0.37580984 0.12711234 0.12694029]
[0.16331629 0.16581909 0.05608595 0.05601004]
oracle solution norm [[0.37163941 0.37001928 0.12854891 0.12979241]]
[[1.63681102 1.6296755  0.56616782 0.57164455]]
marginalized stationary distribution [0.74165869 0.25834131]


In [6]:


[ stateChangeHist, stateHist, a_s, s_a_sprime, p_infty_b_su_long, distr_hist ] =  simulate_rollouts( 
    nS, nA, P, Pi, state_dist, 10000 )
p_infty_b_s = (reshape_byxrow(p_infty_b_su_long.T, nU).T).flatten()


[ p_a1_su, joint_s_a_sprime, s_a_giv_sprime ] = get_auxiliary_info_from_traj(stateChangeHist, 
                                        stateHist, a_s, s_a_sprime, p_infty_b_su_long, distr_hist, nA,nS)
# agg history and process
[aggStateHist, p_a1_s, p_e_s, agg_s_a_sprime, joint_s_a_sprime_agg, s_a_giv_sprime_agg] = agg_history(
            stateHist, s_a_sprime, p_infty_b_s, a_s, p_e_su, nA, nS, nSmarg, nU)

[ stateChangeHist_e, stateHist_e, a_s, s_a_sprime_e, p_infty_e_long, distr_hist_e ] =  simulate_rollouts( 
    nS, nA, P, p_e_su, state_dist, 10000 )

print p_infty_b_su_long
print p_infty_e_long

[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[[0.20077992 0.20417958 0.29567043 0.29937006]]
[[0.33166683 0.33446655 0.16578342 0.16808319]]


## Get biased transitions

In [7]:
joint_s_a_sprime_agg

array([[[0.0185, 0.1805],
        [0.1574, 0.0486]],

       [[0.0181, 0.2839],
        [0.211 , 0.082 ]]])

## Get marginalized transitions and solve for model

In [8]:
def get_transitions(joint_s_a_sprime_agg, p_a1_s, p_b_infty_s,nS,nA): 
    P = np.zeros([nS,nA, nS])
    for s in range(P.shape[0]): 
        for a in range(nA): 
            P[s,a,:] = joint_s_a_sprime_agg[s,a,:] / (p_a1_s[a,s]*p_b_infty_s[s])
    return P
P_marg = get_transitions(joint_s_a_sprime_agg, p_a1_s, p_infty_b_s,nSmarg,nA)
P_marg_mdptoolbox = np.zeros([ nA, nSmarg, nSmarg ])
for a_ind in range(nA): 
    for i_s in range(nSmarg): 
        for i_sprime in range(nSmarg): 
            P_marg_mdptoolbox[ a_ind, i_s, i_sprime] = P_marg[a_ind, i_s, i_sprime]
P_marg


array([[[0.09295116, 0.9069019 ],
        [0.76433643, 0.23600223]],

       [[0.05993977, 0.94016023],
        [0.71996281, 0.27979597]]])

In [9]:
from copy import deepcopy
npi = 8

# pu1 = 1/6
# pu2 = 1/8
nSmarg = int(nS / nU)
pis1u1 = 1.0/4
pis2u1 = 1.0/4
p0 = 0.25
p_e_s = np.asarray([[ p0 ,1-p0 ], [1-p0,p0]])
p_e_su = np.asarray([[ p0 ,p0,1-p0,1-p0 ], [1-p0,1-p0,p0, p0]])


pi_s1u1s = np.linspace(1.0/10, 0.45, npi)
bias = np.zeros(npi)
tight = True; quiet = True; sense_min = False
ngams = 30
logGams_full = np.linspace(0.1, 2.75, ngams)
min_bnds = [None] * ngams; w_min_bnds = [None] * ngams
max_bnds = [None] * ngams; w_max_bnds = [None] * ngams
ms_min = [None] * ngams; ms_max = [None] * ngams; 
w_min_bnds_ = [None] * npi; w_max_bnds_ = [None] * npi; 
min_bnds_ = [None] * npi; max_bnds_ = [None] * npi; 


phi = np.asarray([1,2])

for indpi,param in enumerate(pi_s1u1s): 
    pu1 = param; pu2 = param
    [P,Pi] = get_P(pu1, pu2, pis1u1, pis2u1)
    print P[:,0,:], P[:,1,:]
    
    # simulate
    [ stateChangeHist, stateHist, a_s, s_a_sprime, p_infty_b_su_long, distr_hist ] =  simulate_rollouts( 
        nS, nA, P, Pi, state_dist, 10000 )
    p_infty_b_s = (reshape_byxrow(p_infty_b_su_long.T, nU).T).flatten()
    [ p_a1_su, joint_s_a_sprime, s_a_giv_sprime ] = get_auxiliary_info_from_traj(stateChangeHist, 
                                            stateHist, a_s, s_a_sprime, p_infty_b_su_long, distr_hist, nA,nS)
    # agg history and process
    [aggStateHist, p_a1_s, p_e_s, agg_s_a_sprime, joint_s_a_sprime_agg, s_a_giv_sprime_agg] = agg_history(
                stateHist, s_a_sprime, p_infty_b_s, a_s, p_e_su, nA, nS, nSmarg, nU)

    print p_infty_b_s
    
    
    # Simulate from evaluation policy: 
    [ stateChangeHist_e, stateHist_e, a_s_e, s_a_sprime_e, p_infty_e, distr_hist_e ] =  simulate_rollouts( 
        nS, nA, P, p_e_su, state_dist, n )
    print 'simulated value,', np.dot( (reshape_byxrow(p_infty_e.T, nU).T).flatten(), phi) 
    
    for ind,logGam in enumerate(logGams_full): 
        sense_min = False
        [a_bnd, b_bnd] = get_bnds_as( p_a1_s, logGam )
        [objVal, w_, m] = primal_opt_outer_L1_test_function(gamma, phi, a_bnd,b_bnd, s_a_giv_sprime_agg, p_infty_b_s, p_e_s, p_a1_s, nSmarg, nA, tight, sense_min, quiet)

        print objVal
        print w_
        ms_min[ind] = m 
        min_bnds[ind] = objVal; w_min_bnds[ind] = w_; 
        sense_min = True
        [objVal, w_, m] = primal_opt_outer_L1_test_function(gamma, phi, a_bnd,b_bnd, s_a_giv_sprime_agg, p_infty_b_s, p_e_s, p_a1_s, nSmarg, nA, tight, sense_min, quiet)
        print objVal
        max_bnds[ind] = objVal; w_max_bnds[ind] = w_; 
        ms_max[ind] = m 
        print w_  

    w_min_bnds_[indpi] = deepcopy(w_min_bnds); w_max_bnds_[indpi] = deepcopy(w_max_bnds)
    min_bnds_[indpi] = deepcopy(min_bnds); max_bnds_[indpi] = deepcopy(max_bnds)

[[0.1 0.1 0.4 0.4]
 [0.4 0.4 0.1 0.1]
 [0.1 0.1 0.4 0.4]
 [0.4 0.4 0.1 0.1]] [[0.4 0.4 0.1 0.1]
 [0.1 0.1 0.4 0.4]
 [0.4 0.4 0.1 0.1]
 [0.1 0.1 0.4 0.4]]
[0.25 0.25 0.25 0.25]
[0.6540346 0.3459654]
[0.25 0.25 0.25 0.25]
simulated value, 1.5005749712514376
1.36510737327
[0.9707324812563278, 1.055329144538254]
1.3301001029
[1.0242575861376138, 0.9541419448190372]
1.38362307549
[0.9424225075664808, 1.1088480861293781]
1.31631712557
[1.0453313602125445, 0.9143027667195794]
1.40393972897
[0.9113589314491677, 1.167572609650576]
1.30375013308
[1.0645459286158887, 0.8779783470877084]
1.42623560911
[0.877269174936965, 1.2320180135078935]
1.29229086002
[1.0820668260038577, 0.8448557488753657]
1.45070726866
[0.8398527145952123, 1.3027524259632128]
1.28184084579
[1.098044595822, 0.8146503753549994]
1.47757153914
[0.7987780212603536, 1.3804025904439385]
1.27231051896
[1.1126161901669556, 0.7871033237335096]
1.50369025968
[0.7588432522488072, 1.4558977708209686]
1.26361837185
[1.1259062319387851, 0.

1.69115539942
[0.5602675222908698, 1.5401615752251414]
1.25150412862
[1.3578282622292082, 0.560448482693934]
1.69438536282
[0.5544081237919314, 1.547359183051489]
1.24887693869
[1.362594183965156, 0.5545940872994866]
1.69733298883
[0.5490609067057344, 1.553927633986472]
1.24647941327
[1.366943476844542, 0.5492514732968004]
1.70002297861
[0.5441810612967418, 1.5599219717181512]
1.24429145669
[1.3709125959746742, 0.5443758597129282]
1.70247786864
[0.5397277046541562, 1.5653924162748747]
1.24229473477
[1.3745348009338758, 0.5399263909205757]
1.70471822107
[0.535663535484011, 1.5703847880741195]
1.24047251992
[1.3778404368325592, 0.5358657913863862]
1.70676279716
[0.531954519430007, 1.5749408944702254]
1.23880954993
[1.3808571904948679, 0.5321600509807919]
1.70862871544
[0.5285696021930381, 1.579098882154586]
1.23729189919
[1.3836103239896365, 0.5287781381116612]
1.71033159588
[0.5254804479685066, 1.5828935584557986]
1.23590686142
[1.3861228875354135, 0.5256917381945779]
[[0.25 0.25 0.25 0

[0.40165983 0.59834017]
[0.25 0.25 0.25 0.25]
simulated value, 1.4958752062396878
1.61716621898
[0.9531293611994611, 1.0314637961333162]
1.57725007909
[1.0525073335971706, 0.9647523464137977]
1.63291650175
[0.9139163719291334, 1.0577870878944973]
1.5561888568
[1.1049428038666718, 0.9295529339685127]
1.64729432034
[0.8781203640190688, 1.0818165938728945]
1.53311942243
[1.1623780573231295, 0.8909972165329193]
1.66041903221
[0.8454441769617428, 1.1037517949773863]
1.50785102501
[1.2252880007083828, 0.8487663939094964]
1.67239965446
[0.8156163942524022, 1.1237748904224767]
1.48763932413
[1.2756084439614874, 0.8149867781762542]
1.68333575306
[0.7883891296040696, 1.1420522838202627]
1.47133093131
[1.3162109424902315, 0.7877307225963804]
1.69331825667
[0.7635360007684241, 1.158735943334432]
1.45644183618
[1.353279859683739, 0.7628467252089608]
1.7024302013
[0.7408502755286893, 1.173964646256894]
1.44284890849
[1.3871217491263523, 0.7401289996255752]
1.71074741161
[0.7201431756213401, 1.187865

1.43960538014
[1.8675463489626734, 0.628070486680431]
1.81522239473
[0.615781682887753, 1.164719881379122]
1.43198334866
[1.8929471942734848, 0.6171807814264674]
1.81820053397
[0.6058568676239154, 1.1689747914658042]
1.42502886695
[1.9161233927307155, 0.6072448140593032]
1.8209179509
[0.5968009240461342, 1.1728572038482217]
1.41868331604
[1.937270295319819, 0.5981788348207461]
1.82339752205
[0.5885376147875011, 1.1763998025746727]
1.41289325014
[1.9565660131025067, 0.5899064849541967]
1.82566010811
[0.5809974204687667, 1.179632391596176]
1.40760993582
[1.9741729529772576, 0.5823581383021786]
1.82772473296
[0.5741169429207914, 1.1825821506135292]
1.40278893291
[1.9902392142463576, 0.5754703025780973]
1.82960874662
[0.5678383622191167, 1.1852738678543473]
1.39838971334
[2.004899859000263, 0.5691850747343158]
1.83132797354
[0.5621089425459194, 1.1877301519170993]
1.39437531425
[2.0182780680493453, 0.5634496453977021]
1.83289684754
[0.5568805823781078, 1.1899716246119]
1.39071202228
[2.030

In [ ]:
p_e_su = np.asarray([ p0 ,1-p0 ])



In [ ]:
min_bnds[m_]

In [ ]:
plt.figure(figsize=(3,3))
for m_ in range(npi): 
    
    if m_==0: 
        plt.plot(np.exp(logGams_full),min_bnds_[m_],color=((m_)*1.0/(npi), 0.3,0),label=r'$p_{u_1}=$'+str(np.round(pi_s1u1s[m_],2)) if m_%2 ==0 else None ) 
    else:
        plt.plot(np.exp(logGams_full),min_bnds_[m_],color=((m_)*1.0/(npi), 0.3,0),label=np.round(pi_s1u1s[m_],2) if m_%2 ==0 else None ) 
    plt.plot(np.exp(logGams_full),max_bnds_[m_],color=((m_)*1.0/(npi), 0.3,0))
plt.axvline(3,linestyle='--',color='black',linewidth=1)
plt.plot(3,1.5,marker='*',markersize=12, color='black')
plt.xlabel(r'$\Gamma$ parameter')
plt.ylabel(r'${R}_e$')
plt.legend()
# plt.savefig('conf-randomwalk-varyingtransitions.pdf',bbox_inches='tight')

In [ ]:
R_u0 = np.zeros([nS,nA]); 
for a in range(nA): 
    R_u0[:,a] = R_u0_vec

In [ ]:
[(1 + pis2u2 - 2*pu1 + pis2u1*(-1 + 4*pu1) + 2*pu2 - 4*pis2u2*pu2)/(2.*(2 + pis1u1 - pis2u1 + pis2u2 - 4*pis1u1*pu1 + 4*pis2u1*pu1 -  4*pis2u2*pu2 + pis1u2*(-1 + 4*pu2))),
 (1 + pis2u2 - 2*pu1 + pis2u1*(-1 + 4*pu1) + 2*pu2 - 4*pis2u2*pu2)/(2.*(2 + pis1u1 - pis2u1 + pis2u2 - 4*pis1u1*pu1 + 4*pis2u1*pu1 - 4*pis2u2*pu2 + pis1u2*(-1 + 4*pu2))),
1.0/(2 + (2*(1 + pis2u2 - 2*pu1 + pis2u1*(-1 + 4*pu1) + 2*pu2 - 4*pis2u2*pu2))/(1 + pis1u1 + 2*pu1 - 4*pis1u1*pu1 - 2*pu2 + pis1u2*(-1 + 4*pu2))),
1.0/(2 + (2*(1 + pis2u2 - 2*pu1 + pis2u1*(-1 + 4*pu1) + 2*pu2 - 4*pis2u2*pu2))/(1 + pis1u1 + 2*pu1 - 4*pis1u1*pu1 - 2*pu2 + pis1u2*(-1 + 4*pu2)))]
